In [2]:
import sys
sys.path.append('../src')

from DEFCM import DEFCM
from keras.initializers import VarianceScaling

defcm = DEFCM(
    [784, 500, 500, 2000, 10],
    n_clusters=4,
    fuzzifier=1.5,
    init=VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform'),
    exp_name='mnist',
    cluster_csv_epochs=[0, 3, 5, 10, 15, 20, 25, 30, 50]
)
defcm.model.compile(optimizer='sgd', loss='kld')
defcm.model.summary()
defcm.model.trainable_weights[-1]
defcm.model.get_layer(name='encoder_0')()
# defcm.encoder.summary()

Model: "deep_fuzzy_c_mean_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder_0 (Dense)           (None, 500)               392500    
                                                                 
 encoder_1 (Dense)           (None, 500)               250500    
                                                                 
 encoder_2 (Dense)           (None, 2000)              1002000   
                                                                 
 encoder_3 (Dense)           (None, 10)                20010     
                                                                 
 clustering (ClusteringLayer  (None, 4)                40        
 )                                                               
                                         

ValueError: The first argument to `Layer.call` must always be passed.

In [14]:
import numpy as np

[index, index_array, batch_size] = [0, np.arange(10000), 256]
idx = index_array[index * batch_size: min((index+1) * batch_size, 10000)]

with open('x.npy', 'rb') as f:
    x = np.load(f)[idx]
with open('v.npy', 'rb') as f:
    v = np.load(f)
with open('u.npy', 'rb') as f:
    u = np.load(f)[idx]
print(x.shape, v.shape, u.shape)

A = 2
M = 2

# x[0].gradient
print((2 * A) * (u[0][0] * (x[0] - v[0]) + u[0][1] * (x[0] - v[1]) + u[0][2] * (x[0] - v[2]) + u[0][3] * (x[0] - v[3])))

print("\n", "="*20, "Expected", "="*20)
x_grad = []
for i in range(len(x)):
    xi_grad = np.zeros(len(x[i]))
    for j in range(len(v)):
        xi_grad += (2 * A) * ((u[i][j] ** M) * (x[i] - v[j]))
    x_grad.append(xi_grad)
x_grad = np.array(x_grad)
# print(x_grad, x_grad.shape)

v_grad = []
for j in range(len(v)):
    vj_grad = np.zeros(len(v[j]))
    for i in range(len(x)):
        vj_grad += (-2 * A) * ((u[i][j] ** M) * (x[i] - v[j]))
    v_grad.append(vj_grad)
v_grad = np.array(v_grad)
print(v_grad, v_grad.shape)


print("\n", "="*20, "Actual", "="*20)
# print(x[0], v[0], x[0]-v[0])
res_xgrad = np.sum(
    (np.expand_dims(x, axis=1) - v) *
    (np.expand_dims(u, axis=2) ** M),
    axis=1
) * (2 * A)

res_vgrad = np.sum(
    (np.expand_dims(x, axis=1) - v) *
    (np.expand_dims(u, axis=2) ** M),
    axis=0
) * (-2 * A)
print(res_vgrad, res_vgrad.shape)



(256, 10) (4, 10) (256, 4)
[ 14.518335    8.717884   -6.0188384 -16.411516   22.671898  -47.46149
 -22.175827    1.8444552  11.686701   32.449303 ]

 ==================== Expected ====================
[[  24.46335138   11.33052788   47.10293581  -52.27170717   46.49951947
   -79.79020945  -56.96634705  -75.18557508   -6.40176858   52.79765449]
 [  42.44870482   10.77017418   80.30745989   10.38068027  -56.40830865
    28.13683835  -29.53717304  -24.03682215  109.16593146   -7.13647755]
 [ -43.51135287 -113.43339923   21.94235152  -54.74745766   53.55573999
    20.25483167   11.6120884   -10.76499788   39.40637443  -20.29260363]
 [ -17.18261114   45.91601235  -47.56574884   58.45004875  -34.76463753
    -0.55104965  -11.10671572   91.80241544   13.50807245   10.37263683]] (4, 10)

 ==================== Actual ====================
[[  24.463352     11.330525     47.102943    -52.271683     46.49951
   -79.79021     -56.966354    -75.18558      -6.4017696    52.797665  ]
 [  42.44871     

In [63]:
import numpy as np

[index, index_array, batch_size] = [0, np.arange(10000), 5]
idx = index_array[index * batch_size: min((index+1) * batch_size, 10000)]

with open('x.npy', 'rb') as f:
    x = np.load(f)[idx]
with open('v.npy', 'rb') as f:
    v = np.load(f)
with open('u.npy', 'rb') as f:
    u = np.load(f)[idx]

(k, i) = (3, 1)
A = 2
M = 2

print(x[k])
print(v[i])

print("\n", "="*20, "Actual", "="*20)
u2 = np.power(np.linalg.norm(np.expand_dims(x, axis=1) - v, ord=A, axis=2), 2)
print(u2)
print(np.sum(u2, axis=1))
# print(np.sum(u2, axis=1) / np.transpose(u2))
u2 = np.sum(u2, axis=1) / np.transpose(u2)
print(u2)
# u2 = np.power(u2, M)
# print(u2)

print("\n", "="*20, "Expected", "="*20)
print(u)
# print(x[k]-v[i])
# ans = 0
# for c in x[k] - v[i]:
#     ans += abs(c)**A
# ans = (ans ** (1 / A)) ** 2
# print(ans)

# print(x[0]-v[0])
# print(np.square(x[0]-v[0]))

# for i in range(len(x)):
#     for j in range(len(v)):
#         assert np.sum(np.square(x[i]-v[j])) == np.sum(np.square(np.expand_dims(x, axis=1) - v), axis=2)[i][j]
# print(np.sum(np.square(x[0]-v[1])))
# print(np.sum(np.square(np.expand_dims(x, axis=1) - v), axis=2)[0][1])
# res = np.sum(np.square(np.expand_dims(x, axis=1) - v), axis=2)
# res = res * u
# for i in range(len(x)):
#     for j in range(len(v)):
#         assert np.sum(np.square(x[i]-v[j])) * u[i][j] == res[i][j]
# print(u.shape, res.shape)
# a = 0
# for i in range(len(x)):
#     for j in range(len(v)):
#         a += res[i][j]
# print(a, np.sum(res))

[ 0.14305414 -1.4792433   0.57136184  2.0629823  -0.67780155  2.0009773
 -0.70313257  0.17708838  0.56900716 -0.01598199]
[-0.06503943  1.3026975  -1.5291356  -1.7791566   1.0981764   1.1783146
  2.4597292   0.31787872 -2.148571   -1.0661732 ]

 ==================== Actual ====================
[[38.116707  45.07047   30.032976  10.868708  31.967407  46.567657
  38.235126  58.784767  16.50217   26.020327 ]
 [17.985493   1.4505979 29.289333  19.467289  31.249578  28.705423
  29.471197  38.939823  28.9861    37.403328 ]
 [74.117805  65.01541   58.112457  63.41257   43.415283  60.544014
  33.161076  23.442572  58.86174   58.94649  ]
 [32.347286  49.29914   15.170388  24.806194  24.23558   17.207113
  20.712471  28.446491  24.95093    4.8252378]
 [29.762913  44.971497  15.844978  21.768711  17.282204  38.20786
   3.726245   8.971681  36.417953  18.721329 ]]
[342.16632 262.94818 539.0294  242.00082 235.67537]
[[  8.976807   14.620015    7.2726035   7.4813333   7.918424 ]
 [  7.5918074 181.26

In [43]:
import tensorflow as tf
l = np.sum(res)
l = tf.constant([l])
l
print(u.shape)
# tf.ensure_shape(l, shape=(1,))

(10000, 4)
